## Steps that I used to perform model training, evaluating and inferencing the model using Tao toolkit on AWS EC2 cloud setup



It is expected that the TAO toolkit,the related dependecies and pre-trained models are already installed on the setup, instructions are given in below document:

#### TAO Quick start guide
https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_quick_start_guide.html 


#### Yolo_v4_tiny model

https://docs.nvidia.com/tao/tao-toolkit/text/object_detection/yolo_v4.html


## The below steps were performed in a similar manner for all the three approaches.

### Step 1 TAO dependencies

It is expected that the TAO toolkit and the related dependecies are already installed on the setup, instructions are given in below document:
https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_quick_start_guide.html 

### Step 2 Pre-processing data-set

Create below directory structure to store data and configuration files

USER_EXPERIMENT_DIR=/workspace/tao-experiments/ppe_detector_approach/yolo_v4_tiny

DATA_DOWNLOAD_DIR=/workspace/tao-experiments/ppe_detector_approach/yolo_v4_tiny/data

SPECS_DIR=/workspace/tao-experiments/ppe_detector_approach/yolo_v4_tiny/specs

TRAINING_DIR=/workspace/tao-experiments/ppe_detector_approach/yolo_v4_tiny/training




### Step 3 Data directory

DATA_DOWNLOAD_DIR, this directory will contain below subfolder:

1) train:

- This folder contains images and labels directory which will be used for model training

2) test:

- This folder contains images and labels directory which will be used for model evaluation on test dataset

3) val:

- This folder contains images and labels directory which will be used for model evaluation on validation dataset

4) tfrecords:

- This folder contains tfrecord files for train, test and validation dataset



SPECS_DIR=/workspace/tao-experiments/ppe_detector_approach/yolo_v4_tiny/specs

SPECS_DIR, this directory will contain below files:

- training.cfg

- inference.cfg

- train_convert.cfg

- test_convert.cfg

- val_convert.cfg



### Step 3 Tfrecords generate

Create train_convert.cfg, test_convert.cfg and val_convert.cfg files for TAO to identify folders of dataset that needs to be converted to tfrecords

Configuration file for dataset convert: https://docs.nvidia.com/tao/tao-toolkit/text/object_detection/yolo_v4_tiny.html#preparing-the-input-data-structure

Sample command to generate training data-set tfrecord:
>>  tao yolo_v4 dataset_convert -d /workspace/tao-experiments/ppe_detector_approach_1/yolo_v4_tiny/specs/train_convert.cfg -o /workspace/tao-experiments/ppe_detector_approach_1/yolo_v4_tiny/data/tfrecords/train/ -v


Similarly for validation dataset:
>>  tao yolo_v4 dataset_convert -d /workspace/tao-experiments/ppe_detector_approach_1/yolo_v4_tiny/specs/val_convert.cfg -o /workspace/tao-experiments/ppe_detector_approach_1/yolo_v4_tiny/data/tfrecords/val/ -v

Similarly for Test dataset:
>>  tao yolo_v4 dataset_convert -d /workspace/tao-experiments/ppe_detector_approach_1/yolo_v4_tiny/specs/test_convert.cfg -o /workspace/tao-experiments/ppe_detector_approach_1/yolo_v4_tiny/data/tfrecords/test/ -v


### Step 4: Create training configuration file

Training configuration document: https://docs.nvidia.com/tao/tao-toolkit/text/object_detection/yolo_v4_tiny.html#creating-a-configuration-file

training.cfg file can be found under specs directory

SPECS_DIR=/workspace/tao-experiments/ppe_detector_approach/yolo_v4_tiny/specs




### Step 5: Generating Yolo anchor shapes

Document: https://docs.nvidia.com/tao/tao-toolkit/text/object_detection/yolo_v4.html#generate-anchor-shape

Sample command:
>> tao yolo_v4_tiny kmeans -l /workspace/tao-experiments/ppe_detector_approach_1/yolo_v4_tiny/data/training/labels -i /workspace/tao-experiments/ppe_detector_approach_1/yolo_v4_tiny/data/training/images -x 960 -y 544


### Step 5 Model training

Training model: https://docs.nvidia.com/tao/tao-toolkit/text/object_detection/yolo_v4.html#training-the-model


Sample command used for running model training:
>> tao yolo_v4_tiny train -e /workspace/tao-experiments/ppe_detector_approach_1/yolo_v4_tiny/specs/training.cfg -r /workspace/tao-experiments/ppe_detector_approach_1/yolo_v4_tiny/ppe_training/ -k tlt_encode --gpus 4 --log_file /workspace/tao-experiments/ppe_detector_approach_1/yolo_v4_tiny/ppe_training/training_output.log


In [2]:
# Output of above model training after 100/100 epochs:
# Start to calculate AP for each class
# *******************************
# helmet        AP    0.70701
# person        AP    0.55276
# vest          AP    0.61022
#               mAP   0.62333
# *******************************

At the end of training, tao will generate model.tlt file that can be used for model evaluation.

### Step 6 Model Evaluation

Model evaluation config: https://docs.nvidia.com/tao/tao-toolkit/text/object_detection/yolo_v4.html#evaluating-the-model

Sample command to run model evaluation:
>> tao yolo_v4_tiny evaluate -e /workspace/tao-experiments/ppe_detector_approach_1/yolo_v4_tiny/specs/training.cfg -m /workspace/tao-experiments/ppe_detector_approach_1/yolo_v4_tiny/ppe_training/weights/model.tlt -k tlt_encode


In [1]:
# Evaluate Model (Val dataset):
# Start to calculate AP for each class
# *******************************
# helmet        AP    0.70701
# person        AP    0.55276
# vest          AP    0.61022
#               mAP   0.62333
# *******************************

# Evaluate on Test dataset:
# Start to calculate AP for each class
# *******************************
# helmet        AP    0.62435
# person        AP    0.56971
# vest          AP    0.61299
#               mAP   0.60235
# *******************************



### Step 7 Model Inference

Document: https://docs.nvidia.com/tao/tao-toolkit/text/object_detection/yolo_v4.html#running-inference-on-a-yolov4-model


Sample command to generate inference files using trained model:

For validation dataste
>> tao detectnet_v2 inference -e /workspace/tao-experiments/ppe_detector_approach_1/yolo_v4_tiny/specs/inference.cfg -i /workspace/tao-experiments/ppe_detector_approach_1/yolo_v4_tiny/data/val/images/ -o /workspace/tao-experiments/ppe_detector_approach_1/yolo_v4_tiny/data/val/labels/ -k tlt_encode

For test dataset:
>> tao detectnet_v2 inference -e /workspace/tao-experimentsppe_detector_approach_1/yolo_v4_tiny/specs/inference.cfg -i /workspace/tao-experiments/ppe_detector_approach_1/yolo_v4_tiny/data/test/images/ -o /workspace/tao-experiments/ppe_detector_approach_1/yolo_v4_tiny/data/test/labels/ -k tlt_encode





The further steps for export the model to run on Deepstream were not performed with Yolo_v4_tiny model. For the demonstration, only detectNet_v2 model was used.